1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2023-12-06 23:12:24--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2023-12-06 23:12:25--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucafae6f2b3a30bc0f4da253b762.dl.dropboxusercontent.com/cd/0/inline/CI4ZcPR7NvPpCnx3smn-lx1TQnpAg6Maadl524cgJ0mihckoTaDFo2E49IxY34uT_45O2QxQR62i33fJr_cDy1AiwG85e2RKuCJPwXWtJbKZUJHM2P8RMHcSeGprqXxwB3U/file# [following]
--2023-12-06 23:12:25--  https://ucafae6f2b3a30bc0f4da253b762.dl.dropboxusercontent.com/cd/0/inline/CI4ZcPR7NvPpCnx3smn-lx1TQnpAg6Maadl524cgJ0mihckoTaDFo2E49IxY34uT_45O2QxQR62i33fJr_cDy1AiwG85e2RKuCJPwXWtJbKZUJH

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [1]:
import numpy as np
import pandas as pd

In [2]:
file_name = "./data/data_000637.txt"
df = pd.read_csv(file_name)

In [3]:
df.shape[0]

1310720

In [4]:
N = 20000
N_df = pd.read_csv(file_name,nrows = N)
N_df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
19995,1,0,27,3869200366,2513,29
19996,1,0,63,3869200366,2517,6
19997,1,0,32,3869200366,2519,5
19998,1,0,17,3869200366,2522,21


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [6]:
# The number of BX in a ORBIT is estimated by finding the max value of BX, 
# as it is assumed this is close to the max value before being reset to 0.

max_bx = N_df['BX_COUNTER'].max() 
print('An estimation of number of BX in an ORBIT is:', max_bx)


An estimation of number of BX in an ORBIT is: 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [7]:
new_df = N_df.copy() 
ns_bx = new_df['BX_COUNTER'] * 25 
ns_orbit = ns_bx * new_df['ORBIT_CNT']
new_df['new_col'] = new_df['TDC_MEAS'] + ns_bx + ns_orbit 


In [8]:
offset = new_df.loc[:,['new_col']] 
new_df['new_col'] = abs(offset.iloc[0,0] - new_df['new_col']) 

In [9]:
time_ser = pd.Series(new_df['new_col'], index = new_df.index) 
print(time_ser)

0                     0
1                     1
2        17314670751802
3        17798320772793
4        37337781621199
              ...      
19995    13445483085978
19996    13832403122655
19997    14025863141004
19998    14316053168545
19999    14316053168524
Name: new_col, Length: 20000, dtype: int64


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [11]:
datetime = pd.to_datetime(time_ser, unit='ns')

max_time = datetime.max()
min_time = datetime.min()

dur = max_time - min_time
dur

Timedelta('2 days 15:47:17.029970826')

In [12]:
hours, remainder = divmod(dur.total_seconds(), 3600)
minutes, seconds = divmod(remainder, 60)

print('The duration of the data is', hours, 'hours,', minutes, 'minutes, and', seconds, 'seconds' )


The duration of the data is 63.0 hours, 47.0 minutes, and 17.02997000000323 seconds


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [13]:
df_gr = N_df.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()

sort_df_gr = df_gr.sort_values(ascending=False)

print('TDC channels with most counts:', sort_df_gr.head(3))


TDC channels with most counts: TDC_CHANNEL
139    1693
64      996
63      969
Name: TDC_CHANNEL, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [14]:
N_df_c = N_df.iloc[:,[1,3]] 
mask_c = N_df_c[N_df_c['FPGA'] > 0] 
num_non_empty = mask_c.drop_duplicates() 
print(len(num_non_empty))

199


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [15]:
df_139 = N_df[N_df['TDC_CHANNEL'] == 139]
N_df_filt = df_139.iloc[:,0:4] 
N_df_single = N_df_filt.drop_duplicates(keep=0) 

print('The number of unique orbits are:', len(N_df_single))

The number of unique orbits are: 31


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [ ]:
# I couldn't figure out this one..

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.